In [ ]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from pandas import date_range

# Определение функции для расчета MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Загрузка данных
file_path = '/content/imputed_filled_P_l_LR1.csv'  # Укажите здесь путь к вашему файлу
data = pd.read_csv(file_path)

# Преобразование временных меток
data['time'] = pd.to_datetime(data['time'], utc=True)
data.set_index('time', inplace=True)

# Создание дополнительных временных признаков с индикатором прогресса
for index in tqdm(data.index, desc="Обработка данных"):
    data.at[index, 'hour'] = index.hour
    data.at[index, 'day_of_week'] = index.dayofweek

# Установка размера тестовой выборки (последние 24 часа)
test_size = 24 * 60 // 5  # Предполагаем, что данные записываются каждые 5 минут

# Разделение данных на обучающую и тестовую выборки
train_data = data.iloc[:-test_size]
test_data = data.iloc[-test_size:]

# Выделение признаков и целевой переменной
X_train = train_data[['hour', 'day_of_week']]
y_train = train_data['P_l']
X_test = test_data[['hour', 'day_of_week']]
y_test = test_data['P_l']

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание и обучение модели SVR
svr_model = SVR()
svr_model.fit(X_train_scaled, y_train)

# Предсказание на тестовой выборке
y_pred = svr_model.predict(X_test_scaled)

# Расчет метрик
mse = mean_squared_error(y_test, y_pred)
mse_sqrt = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Среднеквадратичная ошибка (MSE): ", mse_sqrt)
print("Средняя абсолютная ошибка (MAE): ", mae)
print("Коэффициент детерминации (R^2): ", r2)
print("Средняя абсолютная процентная ошибка (MAPE): ", mape)

# Визуализация фактических и предсказанных значений
plt.figure(figsize=(12, 6))
plt.plot(test_data.index, y_test, label='Фактические значения', color='blue')
plt.plot(test_data.index, y_pred, label='Предсказанные значения', color='red')
plt.title('Фактические и предсказанные значения потребления электроэнергии')
plt.xlabel('Время')
plt.ylabel('Потребление электроэнергии')
plt.legend()
plt.show()

# Создание набора данных для прогнозирования на 1 декабря 2023 года
forecast_date = '2023-12-01'
forecast_time_index = date_range(start=forecast_date, periods=288, freq='5T', tz='UTC')
forecast_data = pd.DataFrame(index=forecast_time_index)
forecast_data['hour'] = forecast_data.index.hour
forecast_data['day_of_week'] = forecast_data.index.dayofweek

# Масштабирование данных для прогнозирования
forecast_data_scaled = scaler.transform(forecast_data[['hour', 'day_of_week']])

# Прогнозирование
forecasted_values = svr_model.predict(forecast_data_scaled)

# Добавление прогнозов в DataFrame
forecast_data['predicted_P_l'] = forecasted_values

# Визуализация прогнозов
plt.figure(figsize=(15, 7))
plt.plot(forecast_data.index, forecast_data['predicted_P_l'], label='Предсказанные значения', color='green')
plt.title('Прогноз потребления электроэнергии на 1 декабря 2023 года')
plt.xlabel('Время')
plt.ylabel('Потребление электроэнергии')
plt.legend()
plt.show()
